<a href="https://colab.research.google.com/github/kaorisugi/diveintocode-ml/blob/master/sprint19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# sprint19 セグメンテーション１


KaggleのTGS Salt Identification Challengeのデータセットを使用し、セグメンテーションを行います。

TGS Salt Identification Challenge | Kaggle

セグメンテーション手法としてU-Net[1]を使います。

[1]O.Ronneberger, P.Fischer, T.Brox. U-Net: Convolutional Networks for Biomedical Image Segmentation. Medical Image Computing and Computer-Assisted Intervention (MICCAI), Springer, LNCS, Vol.9351: 234–241, 2015

https://arxiv.org/pdf/1505.04597.pdf

### 【問題1】学習・推定
以下のKeras実装を使用して学習・推定を行ってください。

zhixuhao/unet: unet for image segmentation

《GPU環境での学習》

大規模なデータセット、大きなモデルになるため、GPUを使用する必要があります。

《新たなデータセットの適用》

公開されている実装で用意されたものとは異なるデータセットを入力するための準備が必要です。

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


####GItHubのUnetのデータをクローンする

In [0]:
#Unet用のディレクトリを作成
%cd /content/drive/My Drive/Colab Notebooks/DIVE/kadai/sprint19/unet

/content/drive/My Drive/Colab Notebooks/DIVE/kadai/sprint19/unet


In [0]:
!git clone "https://github.com/zhixuhao/unet.git"

Cloning into 'unet'...
remote: Enumerating objects: 394, done.
remote: Total 394 (delta 0), reused 0 (delta 0), pack-reused 394
Receiving objects: 100% (394/394), 44.91 MiB | 12.90 MiB/s, done.
Resolving deltas: 100% (80/80), done.
Checking out files: 100% (253/253), done.


In [0]:
#GitHubのUnetのデータをそのまま使ってunetを実行してみる
#gitHubのUnetを実行(!cdでUnetのフォルダに移動しておく)
! python '/content/drive/My Drive/Colab Notebooks/DIVE/kadai/sprint19/unet/main.py'

Using TensorFlow backend.





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/content/drive/My Drive/Colab Notebooks/DIVE/kadai/sprint19/unet/model.py:55: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv10)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/1
Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.
2019-09-23 03:43:40.073413: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2019-09-23 03:43:40.073642: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1e7ca00 executing computations on platform Host. Devices:
2019-09-23 03:43:40.073673: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2019-09-23 03:43:40.076281: I te

タスク
*   kaggleのデータセットをダウンロード
*   実装したコードのパスを書き換える
*   論文を読んだ上で、実装したコードのコードリーディングをする


####kaggleの塩コンペのデータセットをダウンロードする


In [0]:
!mkdir .kaggle

In [0]:
!ls -a

data		   .git     LICENSE   __pycache__      unet_membrane.hdf5
dataPrepare.ipynb  img	    main.py   README.md
data.py		   .kaggle  model.py  trainUnet.ipynb


In [0]:
#所有者のみが使える設定
!chmod 600 '/content/drive/My Drive/Colab Notebooks/DIVE/kadai/sprint19/unet/.kaggle/kaggle.json'

In [0]:
import json

token = {'username':'kaorisugi','key':'3652816250535d833485a35952069a51'}
with open('/content/drive/My Drive/Colab Notebooks/DIVE/kadai/sprint19/unet/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [0]:
!cp .kaggle/kaggle.json /root/.kaggle/kaggle.json

In [0]:
!kaggle competitions download -c tgs-salt-identification-challenge

  0% 0.00/322k [00:00<?, ?B/s]
100% 322k/322k [00:00<00:00, 24.4MB/s]
  0% 0.00/264k [00:00<?, ?B/s]
100% 264k/264k [00:00<00:00, 16.7MB/s]
  0% 0.00/922k [00:00<?, ?B/s]
100% 922k/922k [00:00<00:00, 61.0MB/s]
 94% 154M/163M [00:02<00:00, 53.0MB/s]
100% 163M/163M [00:02<00:00, 59.6MB/s]
 76% 29.0M/37.9M [00:00<00:00, 23.2MB/s]
100% 37.9M/37.9M [00:00<00:00, 51.9MB/s]


In [0]:
!unzip -d test train.zip

In [0]:
!unzip -d test test.zip

#### kaggleデータセットの入力
解凍したkaggleのtrain testデータを、Unet内のコードのPathのフォルダに入れて学習、推定できるようにする（コマンドを使わずgoogle driveで作業してしまいました）

In [7]:
#ローカルで書き換えた.pyファイルを移動
!mv "/content/data.py" "/content/drive/My Drive/Colab Notebooks/DIVE/kadai/sprint19/unet"
!mv "/content/main.py" "/content/drive/My Drive/Colab Notebooks/DIVE/kadai/sprint19/unet"
!mv "/content/model.py" "/content/drive/My Drive/Colab Notebooks/DIVE/kadai/sprint19/unet"

mv: cannot stat '/content/data.py': No such file or directory
mv: cannot stat '/content/main.py': No such file or directory


In [9]:
#unetフォルダに移動して、main.pyファイルを実行
%cd "/content/drive/My Drive/Colab Notebooks/DIVE/kadai/sprint19/unet"
!python "/content/drive/My Drive/Colab Notebooks/DIVE/kadai/sprint19/unet/main.py"

/content/drive/My Drive/Colab Notebooks/DIVE/kadai/sprint19/unet
Using TensorFlow backend.





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/content/drive/My Drive/Colab Notebooks/DIVE/kadai/sprint19/unet/model.py:55: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv10)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/1
/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:709: UserWarning: An input could not be retrieved. It could be because a worker has died.We do not have any information on the lost sample.
  UserWarning)
Found 5195 images belonging to 1 classes.
Found 4000 images belonging to 1 classes.
2019-09-24 02:47:21.959632: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-09-24 02:47:21.96413

### 【問題2】コードリーディング

論文[1]に目を通した上で、上記実装のコードリーディングを行ってください。

In [0]:
#.pyファイルにコメント記載（途中）